In [228]:
from os import listdir
from os.path import isfile, join
import fnmatch
import pandas as pd


# *************** change only model name + file_name(if DP) in the following line ******************** 
#model_path = r"csv_files/decomp_dp_age30_1/test/"
model_path = r"csv_files/decomp_dp_white_1/test/"

# take the csv files in that directory
files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*temp_test_isotonic_4*.csv')]
#files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*age3040_thr_temp_test_isotonic_1*.csv')]
#put dp selected model here
#files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, 'test_isotonic_subgroup*.csv')]
print(files)

['temp_test_isotonic_4.csv']


In [229]:
def different_demographics(demographics):
    y_pred = demographics["y_pred"].values
    y_true = demographics["y_true"].values
    prediction = demographics["calibrated_prediction"].values
    a = result_print(y_true,y_pred, prediction)
    return demographics.shape[0], a


In [230]:
#confusion matrix
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import numpy as np
from sklearn.metrics import matthews_corrcoef
import math


def result_print(y_true,y_pred,prediction):
    #print("Confusion Metrix" , confusion_matrix(y_true, y_pred))
    accuracy = accuracy_score(y_true, y_pred)
    #print("Accuracy Score ", accuracy)
    recall = recall_score(y_true, y_pred)
    #print("recall_score", recall)
    precision = precision_score(y_true, y_pred)
    #print("precision_score", precision)
    f1 = f1_score(y_true, y_pred)
    #print("f1_score", f1)
    auc_roc = roc_auc_score(y_true, prediction)
    #print("roc_auc_score", auc_roc)
    #print("precision_recall_curve", precision_recall_curve(y_true, y_pred))

    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    FPR = ((FP)/(FP+TN))
    FNR = ((FN)/(FN+TP))
    TPR = (TP) / (TP + FN)
    TNR = (TN) / (TN + FP)
    #print("Precision_score (Group 0): ", (TN/(TN+FN)))
    precision_0 = (TN/(TN+FN))
    #print("recall_score (Group 0): ", (TN/(TN+FP)))
    recall_0 = (TN/(TN+FP))
    #print("false_positive_rate: ", FPR)
    #print("false_negative_rate: ", FNR)
    balanced_accuracy = (TPR + TNR) / 2 
    #print("balanced_accuracy", balanced_accuracy)
    #MCC = ((TP*TN) - (FP*FN)) / (math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))) 
    MCC = matthews_corrcoef(y_true, y_pred)
    
    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, prediction)

    y_true_inverse = []
    for i in y_true:
        y_true_inverse.append(1-i)
    y_true_inverse = np.array(y_true_inverse)

    y_pred_inverse = []
    for i in y_pred:
        y_pred_inverse.append(1-i)
    y_pred_inverse = np.array(y_pred_inverse)  
    prediction_inverse = []
    for i in prediction:
        prediction_inverse.append(1-i)
    prediction_inverse = np.array(prediction_inverse)

    precision_inv = precision_score(y_true_inverse, y_pred_inverse)
    #print("precision_score_inv: ", precision_inv)  

    f1_inverse = f1_score(y_true_inverse, y_pred_inverse)
    #print("f1_score (class 0): ", f1_inverse)

    (precisions_inverse, recalls_inverse, thresholds_inverse) = metrics.precision_recall_curve(y_true_inverse, prediction_inverse)

    auc_roc_inverse = roc_auc_score(y_true_inverse, prediction_inverse)
    #print("roc_auc_score (class 0)", auc_roc_inverse)

  

    (precisions_inverse, recalls_inverse, thresholds_inverse) = metrics.precision_recall_curve(y_true_inverse, prediction_inverse)


    auprc = metrics.auc(recalls, precisions)
    #print('PR Score_class1: ', auprc)

    auprc_inverse = metrics.auc(recalls_inverse, precisions_inverse)
    #print('PR Score_class0: ', auprc_inverse)


    result_return = []
    k = 3

    result_return.append(round(recall,k))
    result_return.append(round(precision,k))
    result_return.append(round(auprc,k))
    result_return.append(round(f1,k))
    result_return.append(round(recall_0,k))
    result_return.append(round(precision_0,k))
    result_return.append(round(auprc_inverse,k))
    result_return.append(round(f1_inverse,k))
    result_return.append(round(accuracy,k))
    result_return.append(round(balanced_accuracy,k))
    result_return.append(round(auc_roc,k))
    result_return.append(round(MCC,k))
    #result_return.append(round(auc_roc_inverse,k))
    result_return.append(round(FPR,4))
    result_return.append(round(FNR,k))
    
    

    return result_return



In [231]:
for f in files:
    print(f)
    df = pd.read_csv(model_path + f)
    full_list = []
    
    # whole
    shape, res_metrics = different_demographics(df)
    a = ['whole']+ [shape] + res_metrics
    full_list.append(a)
    
    # Male
    shape, res_metrics = different_demographics(df[df['GENDER'] == 'M'])
    a = ['Male']+ [shape] + res_metrics
    full_list.append(a)
    
    # Female
    shape, res_metrics = different_demographics(df[df['GENDER'] == 'F'])
    a = ['female']+ [shape] + res_metrics
    full_list.append(a)
    
    # White
    shape, res_metrics = different_demographics(df[df['ETHNICITY'].str.contains("WHITE")])
    a = ['White']+ [shape] + res_metrics
    full_list.append(a)
    
    # Black
    shape, res_metrics = different_demographics(df[df['ETHNICITY'].str.contains("BLACK")])
    a = ['Black']+ [shape] + res_metrics
    full_list.append(a)
    
    # Hispanic
    shape, res_metrics = different_demographics(df[df['ETHNICITY'].str.contains("HISPANIC")])
    a = ['Hispanic']+ [shape] + res_metrics
    full_list.append(a)
    
    # Asian
    shape, res_metrics = different_demographics(df[df['ETHNICITY'].str.contains("ASIAN")])
    a = ['Asian']+ [shape] + res_metrics
    full_list.append(a)
    
    # age < 30
    shape, res_metrics = different_demographics(df[df['AGE'] < 30])
    a = ['Age < 30']+ [shape] + res_metrics
    full_list.append(a)
    
    # 30 >= age > 40
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 30) & (df['AGE'] < 40) ])
    a = ['Age [30, 40)']+ [shape] + res_metrics
    full_list.append(a)
    
    # 40 >= age > 50
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 40) & (df['AGE'] < 50) ])
    a = ['Age [40, 50)']+ [shape] + res_metrics
    full_list.append(a)
    
    # 50 >= age > 60
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 50) & (df['AGE'] < 60) ])
    a = ['Age [50, 60)']+ [shape] + res_metrics
    full_list.append(a)
    
    # 60 >= age > 70
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 60) & (df['AGE'] < 70) ])
    a = ['Age [60, 70)']+ [shape] + res_metrics
    full_list.append(a)
    
    # 70 >= age > 80
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 70) & (df['AGE'] < 80) ])
    a = ['Age [70, 80)']+ [shape] + res_metrics
    full_list.append(a)
    
    
    # 80 >= age > 90
    shape, res_metrics = different_demographics(df[(df['AGE'] >= 80) & (df['AGE'] < 90) ])
    a = ['Age [80, 90)']+ [shape] + res_metrics
    full_list.append(a)
    
    # age >= 90
    shape, res_metrics = different_demographics(df[df['AGE'] >= 90])
    a = ['Age >= 90']+ [shape] + res_metrics
    full_list.append(a)
    
    df_full_list = pd.DataFrame(full_list, columns = ["Group", "Test_data", "Recall_C1","Precision_C1","AUC_PR_C1", "F1_C1","Recall_C0","Precision_C0","AUC_PR_C0", "F1_C0", "Accuracy", "Balanced_Accuracy", "AUC_ROC", "MCC", "FPR", "FNR"])
    #print(df_full_list)
    
    df_full_list.to_csv(model_path + 'result_'+ f, index=False)
    print('result of ' + f + ' is posted')


temp_test_isotonic_4.csv
result of temp_test_isotonic_4.csv is posted


In [192]:
# # Mean of Results  --- given that there are three files to find mean and stdev 


files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*result_temp_test_isotonic*.csv')]
print("finding mean and avg of files: ", files)

# read three files
file1 = pd.read_csv(model_path + files[0])
file2 = pd.read_csv(model_path + files[1])
file3 = pd.read_csv(model_path + files[2])

# get column names and Group and Test_data
column_name = list(file1.columns)[2:]
Group = list(file1["Group"])
Test_data = list(file1["Test_data"])

# Drop Group and Test_data (No need to do the average them) and convert them to numpy array 
file1 = file1.drop(columns = ["Group", "Test_data"]).to_numpy()
file2 = file2.drop(columns = ["Group", "Test_data"]).to_numpy()
file3 = file3.drop(columns = ["Group", "Test_data"]).to_numpy()

res = np.stack((file1,file2, file3))

average = pd.DataFrame(np.mean(res, axis=0), columns = column_name)
stdev = pd.DataFrame(np.std(res, axis=0), columns = column_name)


# add Group and Test_data information again
average.insert(0, 'Group', Group)
average.insert(1, 'Test_data', Test_data)

average.to_csv(model_path + '1_AVERAGE.csv', index=False)

stdev.insert(0, 'Group', Group)
stdev.insert(1, 'Test_data', Test_data)

stdev.to_csv(model_path + '2_STDEV.csv', index=False)


finding mean and avg of files:  ['result_temp_test_isotonic_0_1.csv', 'result_temp_test_isotonic_0_2.csv', 'result_temp_test_isotonic_0_3.csv']


In [82]:
# # Mean of Results of 2 files  --- given that there are three files to find mean and stdev 


# files = [f for f in listdir(model_path) if fnmatch.fnmatch(f, '*result_temp_test_isotonic*.csv')]
# print("finding mean and avg of files: ", files)

# # read three files
# file1 = pd.read_csv(model_path + files[0])
# file2 = pd.read_csv(model_path + files[1])

# # get column names and Group and Test_data
# column_name = list(file1.columns)[2:]
# Group = list(file1["Group"])
# Test_data = list(file1["Test_data"])

# # Drop Group and Test_data (No need to do the average them) and convert them to numpy array 
# file1 = file1.drop(columns = ["Group", "Test_data"]).to_numpy()
# file2 = file2.drop(columns = ["Group", "Test_data"]).to_numpy()


# res = np.stack((file1,file2))

# average = pd.DataFrame(np.mean(res, axis=0), columns = column_name)
# stdev = pd.DataFrame(np.std(res, axis=0), columns = column_name)


# # add Group and Test_data information again
# average.insert(0, 'Group', Group)
# average.insert(1, 'Test_data', Test_data)

# average.to_csv(model_path + '1_AVERAGE.csv', index=False)

# stdev.insert(0, 'Group', Group)
# stdev.insert(1, 'Test_data', Test_data)

# stdev.to_csv(model_path + '2_STDEV.csv', index=False)


finding mean and avg of files:  ['result_temp_test_isotonic_1_1.csv', 'result_temp_test_isotonic_1_4.csv']
